# Import libraries

In [1]:
# Import Libraries
import cv2
import numpy as np
import torch

In [2]:
#Set the model
model = torch.hub.load("ultralytics/yolov5",
                       "yolov5l", #we download yolov5 small as a first try, we will progressively enlarge in case we can have more accuracy until the device starts having difficulties processing
                       pretrained = True)

Using cache found in C:\Users\ivanp/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-24 Python-3.9.18 torch-2.2.2 CPU

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


In [75]:
#Create a function with the image path as a parameter and the return will be the image with the objects recognized. 

def detector(image_path):
    # Load image
    frame = cv2.imread(image_path)

    # In case we want to resize image to specified resolution:
    #frame = cv2.resize(frame, resolution)
    # we could also have to add to the parameters of the function resolution = (800x600)

    # This is where the inference happens
    pred = model(frame)
    df = pred.pandas().xyxy[0] 
    df = df[df["confidence"] > 0.5]

    for i in range(df.shape[0]):
        bbox = df.iloc[i][["xmin",
                           "ymin",
                           "xmax",
                           "ymax"]].values.astype(int)

        cv2.rectangle(frame,
                      (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                      (255, 0, 255),
                      2)

        # Show text on bounding boxes
        cv2.putText(frame,
                    f"{df.iloc[i]['name']}:{round(df.iloc[i]['confidence'], 4)}",
                    (bbox[0], bbox[1] - 25), 
                    cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    0.75, 
                    (255,0,0), 
                    1) 

    cv2.imshow("frame", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [77]:
detector("images/image2.jpg")